<a href="https://colab.research.google.com/github/petrov-pa/My_projects/blob/main/IMDB%20classification/%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F_%D1%80%D0%B5%D1%86%D0%B5%D0%BD%D0%B7%D0%B8%D0%B9_%D0%BD%D0%B0_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рецензии на imdb

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Имеются 25000 рецензий пользователей imdb с бинарными метками, посчитанными по оценкам: 0 при оценке < 5 и 1 при оценке >=7.

Полные данные: https://www.kaggle.com/c/word2vec-nlp-tutorial/data

Задача предсказать метку класса рецензии по тексту


In [ ]:
#Загрузим выборку
imdb = pd.read_csv('labeledTrainData.tsv', delimiter='\t')
imdb.shape

(25000, 3)

In [ ]:
imdb.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [ ]:
#Убедимся что классы сбалансированы
imdb.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [ ]:
#Разобъём выборку на обучение и контроль
from sklearn.model_selection import train_test_split
texts_train, texts_test, y_train, y_test = train_test_split(imdb.review.values, imdb.sentiment.values)

In [ ]:
#Векторизуем тексты рецензий
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(sublinear_tf=True, use_idf=True)
X_train = vect.fit_transform(texts_train)
X_test = vect.transform(texts_test)

## Логистическая регрессия

Настроим на векторизованных данных логистическую регрессию и посчитаем AUC:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
clf = LogisticRegression()
clf.fit(X_train, y_train)
print('Точность: ', metrics.accuracy_score(y_test, clf.predict(X_test)))
print('ROC-AUC: ', metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))

Точность:  0.88688
ROC-AUC:  0.9576709400068106


Получаем достаточно хорошее качество модели.

In [ ]:
X_train.shape

(18750, 66479)

У нас получилось очень много признаков. Проверим возможно ли сократить их количество без сильной потери качества

Попробуем отбирать признаки с помощью Лассо:

In [ ]:
clf = LogisticRegression(C=0.15, penalty='l1')
clf.fit(X_train, y_train)
print('Количество признаков: ',np.sum(np.abs(clf.coef_) > 1e-4))
print('Точность: ',metrics.accuracy_score(y_test, clf.predict(X_test)))
print('ROC-AUC: ',metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))

Количество признаков:  107
Точность:  0.81264
ROC-AUC:  0.895861389129995


Качество уменьшилось не очень сильно, но модель обучается намного быстрее. В некоторых случаях мы можем пожертвовать качеством ради скорости

Ещё один способ отбора признаков — рандомизированная логистическая регрессия:

In [ ]:
from sklearn.linear_model import RandomizedLogisticRegression
rlg = RandomizedLogisticRegression(C=0.13)
rlg.fit(X_train, y_train)

/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class RandomizedLogisticRegression is deprecated; The class RandomizedLogisticRegression is deprecated in 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Memory is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Parallel is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWar

/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pi

/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pi

/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pi

/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pi

/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pi

/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pi

/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pi

/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)
/home/user/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function delayed is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pi

RandomizedLogisticRegression(C=0.13, fit_intercept=True, memory=None,
               n_jobs=None, n_resampling=200, normalize=True,
               pre_dispatch='3*n_jobs', random_state=None,
               sample_fraction=0.75, scaling=0.5, selection_threshold=0.25,
               tol=0.001, verbose=False)

In [ ]:
#Посмотрим, сколько признаков отбирается
np.sum(rlg.scores_ > 0)

13

In [ ]:
#Настроим логистическую регрессию на отобранных признаках:
X_train_lasso = X_train[:, rlg.scores_ > 0]
X_test_lasso = X_test[:, rlg.scores_ > 0]

In [ ]:
clf = LogisticRegression(C=1)
clf.fit(X_train_lasso, y_train)
print(metrics.accuracy_score(y_test, clf.predict(X_test_lasso)))
print(metrics.roc_auc_score(y_test, clf.predict_proba(X_test_lasso)[:, 1]))

0.73696
0.8105926603014637


В этот раз качество модели ощутимо уменьшилось. 

## Метод главных компонент

Сделаем 100 синтетических признаков с помощью метода главных компонент:

In [ ]:
from sklearn.decomposition import TruncatedSVD
tsvd = TruncatedSVD(n_components=100)
X_train_pca = tsvd.fit_transform(X_train)
X_test_pca = tsvd.transform(X_test)

Обучим на них логистическую регрессию:

In [ ]:
clf = LogisticRegression()
clf.fit(X_train_pca, y_train)
print(metrics.accuracy_score(y_test, clf.predict(X_test_pca)))
print(metrics.roc_auc_score(y_test, clf.predict_proba(X_test_pca)[:, 1]))

0.85584
0.9370604847898281


По 100 полученных таким способом признакам качество получается не намного хуже, чем по всем 66702

Попробуем обучить на них обучить случайный лес:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train_pca, y_train)
print(metrics.accuracy_score(y_test, clf.predict(X_test_pca)))
print(metrics.roc_auc_score(y_test, clf.predict_proba(X_test_pca)[:, 1]))

0.82512
0.9038591666858699


Признаки, которые даёт метод главных компонент, оптимальны для линейных методов, поэтому логистическая регрессия показывает результаты лучше, чем сложные нелинейные классификаторы.